In [1]:
import datetime
import xgboost
import numpy as np
import pandas as pd
import utils.data_loader as data_loader
import utils.display as display
import utils.vector_haversine_distances as vec_hs_dis
from scipy import stats
from sklearn.metrics.pairwise import haversine_distances
import sklearn.metrics
from hotspot.hotpots_discovery_utils import generate_cube_index, cube_to_coordinate

In [2]:
display.configure_logging()
display.configure_pandas()

In [83]:
def mape_vectorized_v2(a, b):
    mask = a != 0
    return (np.fabs(a - b)/a)[mask].mean()

In [4]:
df_od = data_loader.load_od(scale='full', common=False)
# 筛选出处于bbox中的points
df_od['in_bbox'] = ((113.764635 < df_od['destination_log'])
                     & (df_od['destination_log'] < 114.608972)
                     & (22.454727 < df_od['destination_lat'])
                     & (df_od['destination_lat'] < 22.842654)
                     & (113.764635 < df_od['original_log'])
                     & (df_od['original_log'] < 114.608972)
                     & (22.454727 < df_od['original_lat'])
                     & (df_od['original_lat'] < 22.842654))
df_od = df_od.loc[df_od.in_bbox].reset_index(drop=True)
df_od = generate_cube_index(df_od, m=100, n=200)

demand = df_od.groupby(['original_cube', 'destination_cube']).size().reset_index()
demand = demand.rename(columns={0: 'demand'})

demand = demand.loc[demand['demand'] > 10].reset_index()

17-Jul-20 17:11:16 - Loading data/transaction_201407.csv


In [5]:
df_et_od = data_loader.load_od(scale='full', common=True)
# 筛选出处于bbox中的points
df_et_od['in_bbox'] = ((113.764635 < df_et_od['destination_log'])
                     & (df_et_od['destination_log'] < 114.608972)
                     & (22.454727 < df_et_od['destination_lat'])
                     & (df_et_od['destination_lat'] < 22.842654)
                     & (113.764635 < df_et_od['original_log'])
                     & (df_et_od['original_log'] < 114.608972)
                     & (22.454727 < df_et_od['original_lat'])
                     & (df_et_od['original_lat'] < 22.842654))
df_et_od = df_et_od.loc[df_et_od.in_bbox].reset_index(drop=True)
df_et_od = generate_cube_index(df_et_od, m=100, n=200)

et_demand = df_et_od.groupby(['original_cube', 'destination_cube']).size().reset_index()
et_demand = et_demand.rename(columns={0: 'demand'})

17-Jul-20 17:11:45 - Loading data/transaction_common_201407.csv


In [6]:
df_demands = pd.merge(demand, et_demand, how='left', on=['original_cube', 'destination_cube'], suffixes=('_all', '_et'))
df_demands = df_demands.fillna(0)
df_demands['rate'] = df_demands['demand_et'] / df_demands['demand_all']

In [7]:
df_od['duration'] = (df_od['end_time'] - df_od['begin_time']).dt.total_seconds()
df_od = df_od[['original_cube', 'destination_cube', 'original_log', 'original_lat', 'destination_log',
               'destination_lat', 'duration']].groupby(['original_cube', 'destination_cube']).mean()
df_od.reset_index(inplace=True)

In [17]:
df_od_pairs = pd.merge(df_demands[['original_cube', 'destination_cube']],
                       df_od[['original_cube', 'destination_cube', 'original_log', 'original_lat', 'destination_log',
                              'destination_lat', 'duration']],
                       left_on=['original_cube', 'destination_cube'],
                       right_on=['original_cube', 'destination_cube'])

# df_od_pairs = df_od_pairs.sample(frac=1).reset_index(drop=True)

df_cs, date = data_loader.load_cs(scale='part', date=datetime.datetime(2014, 7, 1))
df_cs = df_cs.loc[~df_cs['cs_name'].isin(['LJDL', 'E04', 'BN0002', 'F11', 'S1', 'S2', 'F12', 'F13', 'F15'])]
df_cs.reset_index(drop=True, inplace=True)

# select drop location and CS location as two array
original_locations = df_od_pairs[['original_lat', 'original_log']].to_numpy()
destination_locations = df_od_pairs[['destination_lat', 'destination_log']].to_numpy()
cs_location = df_cs[['Latitude', 'Longitude']].to_numpy()

# earth radius(km)
AVG_EARTH_RADIUS = 6371.0088

# calculate distance between original/destination location and CS location
original_distances_to_cs = haversine_distances(np.radians(original_locations), np.radians(cs_location)) \
                           * AVG_EARTH_RADIUS
destination_distances_to_cs = haversine_distances(np.radians(destination_locations), np.radians(cs_location)) \
                              * AVG_EARTH_RADIUS
# calculate distance between od locations
od_dis = vec_hs_dis.haversine_np(df_od_pairs['original_log'], df_od_pairs['original_lat'],
                                 df_od_pairs['destination_log'], df_od_pairs['destination_lat'])

df_original_distances_to_cs = pd.DataFrame(original_distances_to_cs)
df_destination_distances_to_cs = pd.DataFrame(destination_distances_to_cs)
df_od_dis = pd.DataFrame(od_dis)

17-Jul-20 17:25:33 - Loading 'C:\Users\hkrep\PycharmProjects\ChargingEventsExtraction\data\ChargingStation'


In [89]:
def evaluate(n=10):
    original_capacity = np.repeat(df_cs['chg_points'].values.reshape(1, -1), df_od_pairs.shape[0], axis=0)
    
    a = df_original_distances_to_cs.values
    a.sort(axis=1)
    a = pd.DataFrame(a, df_original_distances_to_cs.index, df_original_distances_to_cs.columns)
    a = a.iloc[:, :n]
    o_dissorted_capacity = np.take_along_axis(original_capacity, df_original_distances_to_cs.values.argsort(axis=1),
                                              axis=1)
    o_dissorted_capacity = pd.DataFrame(o_dissorted_capacity)
    o_dissorted_capacity = o_dissorted_capacity.iloc[:, :n]
    
    b = df_destination_distances_to_cs.values
    b.sort(axis=1)
    b = pd.DataFrame(b, df_destination_distances_to_cs.index, df_destination_distances_to_cs.columns)
    b = b.iloc[:, :n]
    d_dissorted_capacity = np.take_along_axis(original_capacity, df_destination_distances_to_cs.values.argsort(axis=1),
                                              axis=1)
    d_dissorted_capacity = pd.DataFrame(d_dissorted_capacity)
    d_dissorted_capacity = d_dissorted_capacity.iloc[:, :n]
    
    train_x = pd.concat([df_od_pairs['duration'], od_dis, a, o_dissorted_capacity, b, d_dissorted_capacity],
                        axis=1).iloc[:int(0.7*a.shape[0])]
    test_x = pd.concat([df_od_pairs['duration'], od_dis, a, o_dissorted_capacity, b, d_dissorted_capacity],
                       axis=1).iloc[int(0.7*a.shape[0]):]
    train_y = df_demands['rate'].iloc[:int(0.7*a.shape[0])]
    test_y = df_demands['rate'].iloc[int(0.7*a.shape[0]):]
#     print(train_x.iloc[:3], train_y.iloc[:3])
#     print(test_x.iloc[:3], test_y.iloc[:3])
    gbm = xgboost.XGBRegressor(verbosity=0, n_estimators=100, scale_pos_weight=2,
                               validate_parameters=2, learning_rate=0.05,
                               min_child_weight=7) #这行会有个提示，不用管

    param_grid = {
        'max_depth': [4, 8],
        'gamma': [0, 5,],
        'max_delta_step': [0, 3],
        'subsample': [0.7, 1],
        'colsample_bytree': [0.7, 1],
        'reg_alpha': [0, 0.1],
        'reg_lambda': [1, 3,]
    }
    gs = GridSearchCV(gbm, param_grid=param_grid, cv=5, scoring=make_scorer(lambda x, y: 10 - mape_vectorized_v2(x, y)))

    train_x, val_x, train_y, val_y = train_test_split(
        pd.concat([df_od_pairs['duration'], od_dis, a, o_dissorted_capacity, b, d_dissorted_capacity],axis=1).values,
        df_demands['rate'].values, test_size=0.2)

    gs.fit(train_x, train_y)

    print(gs.best_params_)
    print(gs.best_score_)
    print(gs.best_estimator_)
    print(gs.best_index_)

    gs.best_estimator_.fit(train_x, train_y)
    predict_y = gs.best_estimator_.predict(val_x)
    
    # print(sklearn.metrics.mean_absolute_error(val_y, predict_y), sklearn.metrics.mean_squared_error(val_y, predict_y))
    print('mape:', mape_vectorized_v2(val_y, predict_y))

print(df_od_pairs.shape[0], 'sample')
for i in range(5, 6):
    print(i, 'nearest cs as feature:')
    evaluate(n=i)

106788 sample
5 nearest cs as feature:
{'colsample_bytree': 1, 'gamma': 0, 'max_delta_step': 0, 'max_depth': 8, 'reg_alpha': 0, 'reg_lambda': 1, 'subsample': 1}
9.539238289611852
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=8,
             min_child_weight=7, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=2, subsample=1,
             tree_method='exact', validate_parameters=2, verbosity=0)
73
mape: 0.4590369754097927


C:\Users\hkrep\anaconda3\envs\et\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [66]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer

In [72]:
def evaluate(n=10):
    original_capacity = np.repeat(df_cs['chg_points'].values.reshape(1, -1), df_od_pairs.shape[0], axis=0)

    a = df_original_distances_to_cs.values
    a.sort(axis=1)
    a = pd.DataFrame(a, df_original_distances_to_cs.index, df_original_distances_to_cs.columns)
    a = a.iloc[:, :n]
    o_dissorted_capacity = np.take_along_axis(original_capacity, df_original_distances_to_cs.values.argsort(axis=1),
                                              axis=1)
    o_dissorted_capacity = pd.DataFrame(o_dissorted_capacity)
    o_dissorted_capacity = o_dissorted_capacity.iloc[:, :n]

    b = df_destination_distances_to_cs.values
    b.sort(axis=1)
    b = pd.DataFrame(b, df_destination_distances_to_cs.index, df_destination_distances_to_cs.columns)
    b = b.iloc[:, :n]
    d_dissorted_capacity = np.take_along_axis(original_capacity, df_destination_distances_to_cs.values.argsort(axis=1),
                                              axis=1)
    d_dissorted_capacity = pd.DataFrame(d_dissorted_capacity)
    d_dissorted_capacity = d_dissorted_capacity.iloc[:, :n]

    train_x = pd.concat([df_od_pairs['duration'], od_dis, a, o_dissorted_capacity, b, d_dissorted_capacity],
                        axis=1).iloc[:int(0.7*a.shape[0])]
    test_x = pd.concat([df_od_pairs['duration'], od_dis, a, o_dissorted_capacity, b, d_dissorted_capacity],
                       axis=1).iloc[int(0.7*a.shape[0]):]
    train_y = df_demands['rate'].iloc[:int(0.7*a.shape[0])]
    test_y = df_demands['rate'].iloc[int(0.7*a.shape[0]):]
#     print(train_x.iloc[:3], train_y.iloc[:3])
#     print(test_x.iloc[:3], test_y.iloc[:3])
    gbm = xgboost.XGBRegressor(verbosity=1, max_depth=10, n_estimators=100, scale_pos_weight=2) #这行会有个提示，不用管

    gbm.fit(train_x.values, train_y.values)
    predict_y = gbm.predict(test_x.values)

    print(sklearn.metrics.mean_absolute_error(test_y, predict_y), sklearn.metrics.mean_squared_error(test_y, predict_y))
    print('mape:', mape_vectorized_v2(test_y, predict_y))

print(df_od_pairs.shape[0], 'sample')
for i in range(5, 6):
    print(i, 'nearest cs as feature:')
    evaluate(n=i)


106788 sample
5 nearest cs as feature:
0.04432376258435436 0.003580318491623788
mape: 0.6076186987027307
